In [6]:
from sklearn.cluster import AgglomerativeClustering
from pytesseract import Output
from tabulate import tabulate
import pandas as pd
import numpy as np
import pytesseract
import argparse
import imutils
import cv2

In [2]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
# 	help="path to input image to be OCR'd")
# ap.add_argument("-o", "--output", required=True,
# 	help="path to output CSV file")
# ap.add_argument("-c", "--min-conf", type=int, default=0,
# 	help="minimum confidence value to filter weak text detection")
# ap.add_argument("-d", "--dist-thresh", type=float, default=25.0,
# 	help="distance threshold cutoff for clustering")
# ap.add_argument("-s", "--min-size", type=int, default=2,
# 	help="minimum cluster size (i.e., # of entries in column)")
# args = vars(ap.parse_args())

In [3]:
# set a seed for our random number generator
np.random.seed(42)
# load the input image and convert it to grayscale
image = cv2.imread('raw data\IMG_3153.JPG')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [4]:
# initialize a rectangular kernel that is ~5x wider than it is tall,
# then smooth the image using a 3x3 Gaussian blur and then apply a
# blackhat morphological operator to find dark regions on a light
# background
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (51, 11))
gray = cv2.GaussianBlur(gray, (3, 3), 0)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
# compute the Scharr gradient of the blackhat image and scale the
# result into the range [0, 255]
grad = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
grad = np.absolute(grad)
(minVal, maxVal) = (np.min(grad), np.max(grad))
grad = (grad - minVal) / (maxVal - minVal)
grad = (grad * 255).astype("uint8")
# apply a closing operation using the rectangular kernel to close
# gaps in between characters, apply Otsu's thresholding method, and
# finally a dilation operation to enlarge foreground regions
grad = cv2.morphologyEx(grad, cv2.MORPH_CLOSE, kernel)
thresh = cv2.threshold(grad, 0, 255,
	cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.dilate(thresh, None, iterations=3)
cv2.imshow("Thresh", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# find contours in the thresholded image and grab the largest one,
# which we will assume is the stats table
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
tableCnt = max(cnts, key=cv2.contourArea)
# compute the bounding box coordinates of the stats table and extract
# the table from the input image
(x, y, w, h) = cv2.boundingRect(tableCnt)
table = image[y:y + h, x:x + w]
# show the original input image and extracted table to our screen
cv2.imshow("Input", image)
cv2.imshow("Table", table)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
table

array([[[230, 232, 232],
        [230, 232, 232],
        [228, 233, 232],
        ...,
        [219, 225, 220],
        [228, 234, 229],
        [234, 240, 235]],

       [[230, 232, 232],
        [230, 232, 232],
        [228, 233, 232],
        ...,
        [231, 237, 232],
        [233, 239, 234],
        [229, 235, 230]],

       [[230, 232, 232],
        [230, 232, 232],
        [228, 233, 232],
        ...,
        [233, 238, 236],
        [233, 238, 236],
        [228, 233, 231]],

       ...,

       [[205, 207, 207],
        [205, 207, 207],
        [205, 207, 207],
        ...,
        [197, 203, 202],
        [196, 202, 201],
        [195, 201, 200]],

       [[205, 207, 207],
        [205, 207, 207],
        [205, 207, 207],
        ...,
        [195, 201, 200],
        [196, 202, 201],
        [198, 204, 203]],

       [[205, 207, 207],
        [205, 207, 207],
        [205, 207, 207],
        ...,
        [193, 199, 198],
        [197, 203, 202],
        [201, 207, 206]]

: 